<a href="https://colab.research.google.com/github/LudovicaCi/SentimentAnalysis/blob/main/Classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/DataMining/Dataset/trainingSet.csv')

In [18]:
# Verifica la presenza di valori NaN nelle righe
nan_rows = train_df[train_df.isnull().any(axis=1)]

# Stampa le righe contenenti valori NaN
print("Righe con valori NaN:")
print(nan_rows)

Righe con valori NaN:
      content    rating
44        NaN  positive
70        NaN  positive
303       NaN  positive
311       NaN  positive
382       NaN  positive
...       ...       ...
77960     NaN  negative
77977     NaN  negative
78564     NaN  negative
78610     NaN  negative
79183     NaN  negative

[512 rows x 2 columns]


In [8]:
!pip install nltk

In [12]:
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn import metrics
import matplotlib.pyplot as plt
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('punkt')

# Inizializza lo stemmer
stemmer = EnglishStemmer()

# Aggiorna la lista di stopwords in inglese
stop_words = set(stopwords.words('english'))

# Definisci la funzione di analisi personalizzata
def custom_analyzer(doc):
    # Tokenizzazione
    words = word_tokenize(doc)

    # Rimozione delle stop words e stemming
    stemmed_words = [stemmer.stem(word) for word in words if word.isalpha() and word not in stop_words]

    return stemmed_words


analyzer = CountVectorizer().build_analyzer()
X = train_df['content'].values  #text
y = train_df['rating'].values  #sentiment
target_names = ['negative', 'neutral', 'positive']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
#Pipeline Classifier1
text_clf = Pipeline([
    ('vect', CountVectorizer(analyzer=custom_analyzer, max_features=3000)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

#calculating accuracies in cross-valudation
scores = cross_val_score(text_clf, X, y, cv=5)
print("Accuracy MultinomialNB : %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# prediction in cross-validation
predicted = cross_val_predict(text_clf, X, y, cv=5)


print(metrics.classification_report(y, predicted,
                                    target_names=target_names))  # metrics extractions (precision    recall  f1-score   support)
print(metrics.confusion_matrix(y, predicted))
ConfusionMatrixDisplay.from_predictions(y,predicted)
plt.title("Multinominal Naive Bayes Confusion Matrix")
plt.show()

ValueError: ignored

In [13]:
#Pipeline Classifier2
text_clf2 = Pipeline([
    ('vect', CountVectorizer(analyzer=custom_analyzer, max_features=3000)),
    ('tfidf', TfidfTransformer()),
    ('feat_sel', SelectKBest(chi2,k=100)),
    ('clf', tree.DecisionTreeClassifier(max_leaf_nodes=30)),
])

#calculating accuracies in cross-valudation
scores2 = cross_val_score(text_clf2, X, y, cv=5)
print("Accuracy Decision Tree : %0.2f (+/- %0.2f)" % (scores2.mean(), scores2.std() * 2))

# prediction in cross-validation
predicted = cross_val_predict(text_clf2, X, y, cv=5)

print(metrics.classification_report(y, predicted,
                                    target_names=target_names))  # metrics extractions (precision    recall  f1-score   support)
print(metrics.confusion_matrix(y, predicted))
ConfusionMatrixDisplay.from_predictions(y,predicted)
plt.title("Decision Tree Confusion Matrix")
plt.show()

ValueError: ignored

In [ ]:
# Applica la funzione di pre-processamento alla colonna 'content'
train_df['content_processed'] = train_df['content'].apply(preprocess_text)

# Estrai i testi pre-processati
preprocessed_texts = train_df['content_processed'].tolist()

# Ottieni la matrice TF-IDF
tfidf_matrix = tfidf_transform(preprocessed_texts)